Goal: regressions that work with deductibles (which might be overly amibtious). 

Basically, we have a claim count, which is a Poisson model. Every claim will come from a gamma distribtuion. However, if it is under a certain deductible, we do not code it in. 

Couple of imagined steps: 
- do a Poisson, where claim count is generated randomly, but every claim is ignored with X% probability (this is probably super easy)
- same as above, but make the X% probability depend on a variable, e.g., for every 1k increase, the chance of observing decreas
- make a Gamma with deductible
- combine everything, features determine baseline frequency, then the severity is clipped by a deductible amount, which then might impact frequency

Found some stuff like this one: https://www.stata.com/stata14/censored-poisson/ but it's not the type of censored I am looking for. 

Maybe censored is the wrong word...

In [3]:
import numpy as np
import pandas as pd

# Frequency with Fixed Random Claim Cancel

One binary categorical feature, and each claim is cancelled by a fix percentage. The cancelled ones we don't observe. 

My theory is that this is simply going to be another Poisson where we multiply the lambda with the prob of cancel. Or 1 - prob of cancel...

In [16]:
df = pd.DataFrame.from_dict({'cat_feature' : np.repeat([0,1],10000)})

In [20]:
df['poisson_lambda'] = df.apply(lambda x: 0.2 if x['cat_feature'] == 0 else 0.7, axis = 1)

In [31]:
df['raw_claim_number'] = np.random.poisson(df['poisson_lambda'])

Every raw_claim_number can be cancelled with the same chance. I think I want to generate this from scratch...

In [34]:
df.dtypes

cat_feature           int64
poisson_lambda      float64
claim_number          int64
raw_claim_number      int64
dtype: object

In [35]:
df['claim_cancellation_simulations'] = df.apply(lambda x: np.random.uniform(0,1,x['raw_claim_number']), axis = 1)

TypeError: 'numpy.float64' object cannot be interpreted as an integer